<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#|-Preliminaries" data-toc-modified-id="|-Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>| Preliminaries</a></span></li><li><span><a href="#|-Cleaning" data-toc-modified-id="|-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>| Cleaning</a></span></li><li><span><a href="#|-Named-Entity-Recognition-w/-spaCy" data-toc-modified-id="|-Named-Entity-Recognition-w/-spaCy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>| Named Entity Recognition w/ spaCy</a></span></li><li><span><a href="#|-Gender-Guesser" data-toc-modified-id="|-Gender-Guesser-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>| Gender Guesser</a></span></li><li><span><a href="#|-Group-per-article-and-get-share-of-gender-per-article" data-toc-modified-id="|-Group-per-article-and-get-share-of-gender-per-article-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>| Group per article and get share of gender per article</a></span></li></ul></div>

# | Preliminaries

In [1]:
import os, json
import pandas as pd
import numpy as np
import regex as re

#from bs4 import BeautifulSoup as bs

# SpaCy 
import spacy
import en_core_web_trf

nlp = spacy.load("en_core_web_trf")

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

import warnings
warnings.filterwarnings('ignore') # (action='once')


In [2]:
%%time

'''with open('.../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

df = pd.DataFrame(data)'''


'''filename = ".../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_json(filename)'''

df = pd.read_json(".../Downloads/WashingtonPost.v4/data/TREC_Washington_Post_collection.v4.jl", lines=True,nrows=10000)

CPU times: user 510 ms, sys: 124 ms, total: 634 ms
Wall time: 640 ms


# | Cleaning

In [3]:
with pd.option_context('display.max_colwidth', None):
  display(df[:2])

id  \
0  b2e89334-33f9-11e1-825f-dabc29fd7071   
1  749ec5b2-32f5-11e1-825f-dabc29fd7071   

                                                                                                                                            article_url  \
0  https://www.washingtonpost.com/sports/colleges/danny-coale-jarrett-boykin-are-a-perfect-1-2-punch-for-virginia-tech/2011/12/31/gIQAAaW4SP_story.html   
1                                         https://www.washingtonpost.com/local/bubbly-planet-venus-starts-off-new-year/2011/12/30/gIQAlbm4SP_story.html   

                                                                   title  \
0  Danny Coale, Jarrett Boykin are a perfect 1-2 punch for Virginia Tech   
1                                Bubbly planet Venus starts off New Year   

                      author  published_date  \
0             Mark Giannotto   1325376562000   
1  Blaine P. Friedlander Jr.   1325376729000   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [4]:
def cleaning_for_NER(df):
    
    '''dictionary = {
              "id"                     : "oid",
              "name"                   : "article_title",
              'teaser'                 : 'article_teaser',
              "text"                   : "article_text",
              "regions"                : "cms_regions"
                }
    '''
    df = df.copy()
    #df.rename(columns=dictionary, inplace=True)  # rename columns
    df.drop_duplicates(subset=['id'], inplace=True)

    # Remove "\n" in article_text
    df["content"] = df["content"].apply(lambda x: re.sub("\{'content': '"," ", str(x)))
 
    return df    

In [5]:
# Safe cleaned data in new df
df = cleaning_for_NER(df)

In [6]:
#df.drop(columns = ['contents'], inplace = True)

In [7]:
# drop empty rows
nan_value = float("NaN")
df.replace("", 'no title', inplace=True)
df["title"].replace(np.NaN, 'no title', inplace=True)

#df.dropna(subset = ["title"], inplace=True)

In [8]:
# Combine column "title" & "content" into "merged_total_text":
df['merged_total_text'] = df.apply(lambda r: r['title'] + r['content'], axis=1)

In [36]:
print(df.head(2))

                                     id  \
0  b2e89334-33f9-11e1-825f-dabc29fd7071   
1  b2e89334-33f9-11e1-825f-dabc29fd7071   

                                         article_url  \
0  https://www.washingtonpost.com/sports/colleges...   
1  https://www.washingtonpost.com/sports/colleges...   

                                               title          author  \
0  Danny Coale, Jarrett Boykin are a perfect 1-2 ...  Mark Giannotto   
1  Danny Coale, Jarrett Boykin are a perfect 1-2 ...  Mark Giannotto   

   published_date                                           contents     type  \
0   1325376562000  [{'content': 'Colleges', 'mime': 'text/plain',...  article   
1   1325376562000  [{'content': 'Colleges', 'mime': 'text/plain',...  article   

                source                                            content  \
0  The Washington Post  [ Colleges', 'mime': 'text/plain', 'type': 'ki...   
1  The Washington Post  [ Colleges', 'mime': 'text/plain', 'type': 'ki...   

        

# | Named Entity Recognition w/ spaCy

In [10]:
df_1 = df.copy()

In [11]:
# write entity in parsed_articles 
import tqdm
parsed_articles = []
for article, id in tqdm.tqdm(zip(df_1['content'], df_1['id']), total=len(df_1)):
    parsed_articles.extend([[id, str(entity), entity.label_] for entity  in nlp(article).ents])

100%|██████████████████████████████████| 10000/10000 [15:03:10<00:00,  5.42s/it]


In [12]:
# create dfs from entities
df_2 = pd.DataFrame(parsed_articles, columns=['id', 'content', 'entity_type'])

# rename columns for one df
df_2.rename(columns={'content': 'entity'}, inplace=True)

In [14]:
# merge back to initial df
df = (pd.merge(df_1, df_2, on='id'))

In [15]:
df.to_pickle(r"Washington_Post_NER_all_10K.pkl")

In [18]:
df_ner = df.copy()

In [19]:
# NER 'PERSON' STARTS HERE
df = (df.loc[df['entity_type'].isin(['PERSON'])])

In [35]:
df.head()

,id,article_url,title,author,published_date,contents,type,source,content,merged_total_text,entity,entity_type,first_name,last_name,gender_guesser
0,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny Coale,PERSON,Danny,Coale,male
1,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Jarrett Boykin,PERSON,Jarrett,Boykin,male
4,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Danny Coale,PERSON,Danny,Coale,male
8,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Jarrett Boykin,PERSON,Jarrett,Boykin,male
9,b2e89334-33f9-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,"Danny Coale, Jarrett Boykin are a perfect 1-2 ...",Mark Giannotto,1325376562000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...","Danny Coale, Jarrett Boykin are a perfect 1-2 ...",John Bazemore,PERSON,John,Bazemore,male


In [23]:
df = df[df['author'] != df['entity']] # if author is in col entity -> remove, we only want Person from within the article

In [26]:
df[['first_name', 'last_name']] = df['entity'].str.split(' ', 1, expand=True) # split names
# to do: tri-grams?

# | Gender Guesser

In [28]:
# !pip install gender_guesser    
import gender_guesser.detector as gender
gd = gender.Detector()
df['gender_guesser'] = df['first_name'].apply(str.capitalize).map(lambda x: gd.get_gender(x))

> __unknown__ (name not found), __andy__ (androgynous), __male__, __female__, __mostly_male__, or __mostly_female__. The difference between andy and unknown is that the former is found to have the same probability to be male than to be female, while the later means that the name wasn’t found in the database.

In [34]:
df.gender_guesser.value_counts()

male             73103
unknown          70979
female           19916
mostly_male       4567
mostly_female     3435
andy              2618
Name: gender_guesser, dtype: int64

In [43]:
df.loc[df['gender_guesser'] == 'andy']#[:40]

,id,article_url,title,author,published_date,contents,type,source,content,merged_total_text,entity,entity_type,first_name,last_name,gender_guesser
1593,69f1a5e2-342d-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,Chick-fil-A Bowl: Virginia football falls to A...,Steve Yanda,1325392162000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...",Chick-fil-A Bowl: Virginia football falls to A...,Greer,PERSON,Greer,None,andy
1613,69f1a5e2-342d-11e1-825f-dabc29fd7071,https://www.washingtonpost.com/sports/colleges...,Chick-fil-A Bowl: Virginia football falls to A...,Steve Yanda,1325392162000,"[{'content': 'Colleges', 'mime': 'text/plain',...",article,The Washington Post,"[ Colleges', 'mime': 'text/plain', 'type': 'ki...",Chick-fil-A Bowl: Virginia football falls to A...,London,PERSON,London,None,andy
1655,6033cbfe-21e8-11e1-a34e-71d4bf6b8d0a,https://www.washingtonpost.com/lifestyle/magaz...,Dave Barry’s Year in Review: The 2011 Festival...,Dave Barry,1325394136000,"[{'content': 'Magazine', 'mime': 'text/plain',...",article,The Washington Post,"[ Magazine', 'mime': 'text/plain', 'type': 'ki...",Dave Barry’s Year in Review: The 2011 Festival...,Hu Jintao,PERSON,Hu,Jintao,andy
1660,6033cbfe-21e8-11e1-a34e-71d4bf6b8d0a,https://www.washingtonpost.com/lifestyle/magaz...,Dave Barry’s Year in Review: The 2011 Festival...,Dave Barry,1325394136000,"[{'content': 'Magazine', 'mime': 'text/plain',...",article,The Washington Post,"[ Magazine', 'mime': 'text/plain', 'type': 'ki...",Dave Barry’s Year in Review: The 2011 Festival...,Hu,PERSON,Hu,None,andy
1783,6033cbfe-21e8-11e1-a34e-71d4bf6b8d0a,https://www.washingtonpost.com/lifestyle/magaz...,Dave Barry’s Year in Review: The 2011 Festival...,Dave Barry,1325394136000,"[{'content': 'Magazine', 'mime': 'text/plain',...",article,The Washington Post,"[ Magazine', 'mime': 'text/plain', 'type': 'ki...",Dave Barry’s Year in Review: The 2011 Festival...,bin Laden’s,PERSON,bin,Laden’s,andy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758971,aac33f22-7357-11e1-958a-fd4578434472,https://www.washingtonpost.com/opinions/the-ei...,The Eisenhower memorial design: Get over it or...,no title,1332369238000,"[{'content': 'Letters to the Editor', 'mime': ...",article,The Washington Post,"[ Letters to the Editor', 'mime': 'text/plain'...",The Eisenhower memorial design: Get over it or...,Ike,PERSON,Ike,None,andy
758980,0cd47a3e-7356-11e1-958a-fd4578434472,https://www.washingtonpost.com/opinions/2012/0...,no title,no title,1332369238000,"[{'content': 'Letters to the Editor', 'mime': ...",article,The Washington Post,"[ Letters to the Editor', 'mime': 'text/plain'...","no title[ Letters to the Editor', 'mime': 'tex...",Ike,PERSON,Ike,None,andy
759661,83e82e56-7372-11e1-9ab6-c7562e04789c,https://www.washingtonpost.com/local/documenti...,Documenting D.C.’s punk past — with your help,John Kelly,1332370909000,"[{'content': 'Local', 'mime': 'text/plain', 't...",article,The Washington Post,"[ Local', 'mime': 'text/plain', 'type': 'kicke...",Documenting D.C.’s punk past — with your help[...,Doc Johnson’s,PERSON,Doc,Johnson’s,andy
760698,66414244-739e-11e1-9ab6-c7562e04789c,https://www.washingtonpost.com/sports/redskins...,NFL hits Saints with harsh penalties for bount...,Mark Maske,1332373860000,"[{'content': 'Redskins/NFL', 'mime': 'text/pla...",article,The Washington Post,"[ Redskins/NFL', 'mime': 'text/plain', 'type':...",NFL hits Saints with harsh penalties for bount...,Payton,PERSON,Payton,None,andy


In [38]:
df.first_name.value_counts()

Romney                    6427
Obama                     5922
Gingrich                  2656
Santorum                  2452
John                      2379
                          ... 
Smedley                      1
Mussolini                    1
mahmoud-ahmadinejad?_s       1
Narender                     1
DeMaurice                    1
Name: first_name, Length: 16562, dtype: int64

In [ ]:
# to do:
# Namen cleanen
# weitere lib für gender identification einbauen
# function schreiben: wenn nur Nachname genannt, schaue in anderer Zeile pro Artikel, ob Name schon genannt, dann Vorname auffüllen
# oder
# wenn Obama, Santorum, Gingrich, etc. dann aus erstellter Liste oder via Wikidata mit Vornamen auffüllen
# # dominique rodgers cromartie etc. (famous people)  -> Wikidata? 

# | Group per article and get share of gender per article

In [ ]:
# next